<a href="https://colab.research.google.com/github/Anastasios-K/Classification/blob/master/WineQualClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive/')

In [0]:
from pathlib import Path
path = Path('/content/drive/My Drive/DS Projects/Classification/Wine Quality')
directories = list(path.iterdir())
# ------> Look for further selection from the directories

# Add extra datasets into the parent directory

In [0]:
dimensions = 12
attributes = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
              'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
              'pH', 'sulphates', 'alcohol', 'quality']

# Creating a dictionary of the dataframes
def DF_dict_generator(list_of_datasets, dimensionality, attributes):
  wine_dict = {}
  for i, item in enumerate(list_of_datasets):
    wine_dict["df{0}".format(i+1)]=pd.read_csv(item, delimiter=';', skip_blank_lines=False,skipinitialspace=True)
    if len(wine_dict["df{0}".format(i+1)].columns) != dimensionality:
      wine_dict["df{0}".format(i+1)]=pd.read_csv(item, delimiter=';', skip_blank_lines=False,skipinitialspace=True, skiprows=[0], names=attributes)
    else:
      pass
    wine_dict["df{0}".format(i+1)]['wine type'] = pd.Series(i+1, index=wine_dict["df{0}".format(i+1)].index)
  df = pd.concat(wine_dict, ignore_index=True)
  return(df)

In [0]:
#---------> use of the above function
df = DF_dict_generator(directories,12,attributes)

In [0]:
# Use it to add some noise in the data
# Use it to test the how data quality below works

def Null_test(df):
  import random as rng
  import numpy as np
  rng.seed()
  num_of_NAN = np.round(len(df)*len(df.columns)*0.4, decimals=0)
  for x in range(int(num_of_NAN)):
      df.iloc[rng.randint(0,len(df)-1),rng.randint(0,len(df.columns)-1)]=np.nan
  return(df)
if __name__ == "__main__":
    pass


In [0]:
# THIS IS the quality funcction
# It is in progress
# ROW decison needs to be added
for column in df:
  attr_quality = df[column].isnull().value_counts().iloc[0]/len(df)
  if attr_quality > 0.1:
    df = df.drop([column], axis=1)